In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imc2025-dependencies/imc_2025_deps/models/aliked_lightglue.pth
/kaggle/input/imc2025-dependencies/imc_2025_deps/models/dedode_descriptor_G.pth
/kaggle/input/imc2025-dependencies/imc_2025_deps/models/dedode_detector_L.pth
/kaggle/input/imc2025-dependencies/imc_2025_deps/models/dinov2_vitl14_pretrain.pth
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/.isort.cfg
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/Dockerfile
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/pipeline_SfM.ipynb
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/LICENSE
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/pipeline_Aachen.ipynb
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/.gitignore
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/.gitmodules
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hloc_repo/README.md
/kaggle/input/imc2025-dependencies/imc_2025_deps/repos/hlo

In [9]:
import sys
import os
import shutil
import subprocess
from pathlib import Path

# --- SETUP PATHS ---
DEPS_PATH = Path("/kaggle/input/imc2025-dependencies/imc_2025_deps")
WHEELS_SRC = DEPS_PATH / "wheels"
WHEELS_TEMP = Path("/tmp/safe_wheels")

# --- SMART INSTALLATION ---
if WHEELS_TEMP.exists(): shutil.rmtree(WHEELS_TEMP)
WHEELS_TEMP.mkdir()

# Filter out dangerous packages (torch/nvidia)
for whl in WHEELS_SRC.glob("*.whl"):
    if not any(x in whl.name for x in ['torch-', 'torchvision', 'nvidia-', 'torchaudio']):
        shutil.copy(whl, WHEELS_TEMP / whl.name)

print("📦 Installing dependencies...")
subprocess.run(f"pip install --no-index --find-links {str(WHEELS_TEMP)} yacs einops timm h5py omegaconf scipy tqdm typing_extensions", shell=True)
subprocess.run(f"pip install --no-index --find-links {str(WHEELS_TEMP)} kornia pycolmap --no-deps", shell=True)

# --- CRITICAL FIX: PRE-LOAD CACHE ---
# We manually put the weights exactly where the libraries look for them.
CACHE_DIR = Path("/root/.cache/torch/hub/checkpoints")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print("🛠️ Injecting weights into system cache...")

# 1. Fix DINOv2 (for DeDoDe)
DINO_SRC = DEPS_PATH / "models" / "dinov2_vitl14_pretrain.pth"
DINO_DST = CACHE_DIR / "dinov2_vitl14_pretrain.pth"
if DINO_SRC.exists():
    shutil.copy(DINO_SRC, DINO_DST)
    print("   ✅ DINOv2 cached.")

# 2. Fix LightGlue (The error you just saw)
# Note: The downloader looks for 'aliked_lightglue_v0-1_arxiv.pth', so we rename it during copy.
LG_SRC = DEPS_PATH / "models" / "aliked_lightglue.pth"
LG_DST = CACHE_DIR / "aliked_lightglue_v0-1_arxiv.pth" 
if LG_SRC.exists():
    shutil.copy(LG_SRC, LG_DST)
    print(f"   ✅ LightGlue cached as {LG_DST.name}")
else:
    print("   ⚠️ LightGlue source file not found!")

shutil.rmtree(WHEELS_TEMP)
print("✅ Installation Complete.")

📦 Installing dependencies...
Looking in links: /tmp/safe_wheels
Processing /tmp/safe_wheels/yacs-0.1.8-py3-none-any.whl
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
Processing /tmp/safe_wheels/timm-1.0.22-py3-none-any.whl


ERROR: Could not find a version that satisfies the requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64" (from torch) (from versions: 12.8.93)
ERROR: No matching distribution found for nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"


Looking in links: /tmp/safe_wheels
🛠️ Injecting weights into system cache...
   ✅ DINOv2 cached.
   ✅ LightGlue cached as aliked_lightglue_v0-1_arxiv.pth
✅ Installation Complete.


In [10]:
import sys
import shutil
import torch
import importlib
from pathlib import Path

# --- 1. PATH SETUP ---
DEPS_PATH = Path("/kaggle/input/imc2025-dependencies/imc_2025_deps")
sys.path.insert(0, str(DEPS_PATH / "repos" / "DeDoDe"))
sys.path.insert(0, str(DEPS_PATH / "repos" / "dinov2"))
sys.path.insert(0, str(DEPS_PATH / "repos" / "LightGlue"))

# --- 2. NUCLEAR RESET (Unload old HLOC) ---
modules_to_kill = [k for k in sys.modules.keys() if k.startswith('hloc')]
for k in modules_to_kill: del sys.modules[k]

# --- 3. COPY HLOC & INJECT CODE ---
HLOC_SRC = DEPS_PATH / "repos" / "hloc_repo"
HLOC_DST = Path("/kaggle/working/hloc_lib")
if HLOC_DST.exists(): shutil.rmtree(HLOC_DST)
shutil.copytree(HLOC_SRC, HLOC_DST)
sys.path.insert(0, str(HLOC_DST))

# --- 4. WRITE CORRECTED DEDODE.PY ---
dedode_code = """
import torch
import torch.nn.functional as F
from ..utils.base_model import BaseModel

class DeDoDe(BaseModel):
    default_conf = {
        'detector_model_path': None,
        'descriptor_model_path': None,
        'max_keypoints': 2048,
    }
    required_inputs = ['image']

    def _init(self, conf):
        from DeDoDe.model_zoo import dedode_detector_L, dedode_descriptor_G
        
        det_w = torch.load(conf['detector_model_path'], map_location="cpu")
        desc_w = torch.load(conf['descriptor_model_path'], map_location="cpu")
        
        self.detector = dedode_detector_L(weights=det_w)
        self.descriptor = dedode_descriptor_G(weights=desc_w)
        self.max_keypoints = conf['max_keypoints']

    def _forward(self, data):
        img = data['image']
        B, C, H, W = img.shape
        
        # AUTO-PAD to multiple of 14 (Fixes DINOv2 crash)
        pad_h = (14 - H % 14) % 14
        pad_w = (14 - W % 14) % 14
        if pad_h > 0 or pad_w > 0:
            img = F.pad(img, (0, pad_w, 0, pad_h))
            
        batch = {"image": img}
        
        # Use .detect() (Fixes API mismatch)
        detections = self.detector.detect(batch, num_keypoints=self.max_keypoints)
        kpts = detections["keypoints"]
        scores = detections["confidence"]
        
        descriptors = self.descriptor.describe_keypoints(batch, kpts)["descriptions"]
        
        return {
            'keypoints': kpts, 
            'scores': scores,
            'descriptors': descriptors.transpose(-1, -2), 
        }
"""
with open(HLOC_DST / "hloc" / "extractors" / "dedode.py", "w") as f:
    f.write(dedode_code)

import hloc
import DeDoDe
# Import pairs_from_exhaustive for offline matching
from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive

print(f"✅ Environment Ready. HLOC loaded from: {hloc.__file__}")

✅ Environment Ready. HLOC loaded from: /kaggle/working/hloc_lib/hloc/__init__.py


In [16]:
# --- CONFIGURATION (SIFT BACKUP) ---
# We switch to SIFT because DeDoDe dims (256) don't match our LightGlue weights (128)
# and we are offline so we can't download the correct ones.

# 1. SIFT Config (Standard, robust)
feature_conf = {
    'output': 'feats-sift',
    'model': {
        'name': 'dog', # Difference of Gaussians (SIFT detector)
        'options': {
            'first_octave': -1,
            'peak_threshold': 0.01,
        },
    },
    'preprocessing': {
        'grayscale': True,
        'resize_max': 1600, # SIFT handles larger images well
    },
}

# 2. Matcher Config (Nearest Neighbor + Mutual Check)
# Since we don't have SIFT-specific LightGlue weights, NN-Mutual is the strongest baseline.
matcher_conf = {
    'output': 'matches-nn',
    'model': {
        'name': 'nearest_neighbor',
        'do_mutual_check': True, # Enforce A->B and B->A match
        'distance_threshold': None, 
    }
}

print("✅ Configuration Loaded (Switched to SIFT+NN for safety).")

✅ Configuration Loaded (Switched to SIFT+NN for safety).


In [17]:
from PIL import Image
from sklearn.cluster import DBSCAN
import numpy as np

def get_dino_transforms():
    import torchvision.transforms as T
    return T.Compose([
        T.Resize((518, 518)), 
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

def cluster_dataset(dataset_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    repo_path = str(DEPS_PATH / "repos" / "dinov2")
    model = torch.hub.load(repo_path, 'dinov2_vitl14', source='local', pretrained=False)
    model.load_state_dict(torch.load(WEIGHTS_DIR / "dinov2_vitl14_pretrain.pth", map_location=device))
    model.to(device).eval()
    
    transforms = get_dino_transforms()
    image_paths = sorted(list(dataset_path.glob("**/*.*")))
    image_paths = [x for x in image_paths if x.suffix.lower() in ['.jpg', '.png', '.jpeg']]
    
    if not image_paths: return {}, []

    embeddings = []
    valid_paths = []
    
    print(f"   Extracting DINO embeddings for {len(image_paths)} images...")
    with torch.no_grad():
        for img_p in image_paths:
            try:
                img = Image.open(img_p).convert('RGB')
                img_t = transforms(img).unsqueeze(0).to(device)
                features = model(img_t)
                embeddings.append(features.cpu().numpy().flatten())
                valid_paths.append(img_p)
            except: pass

    if not embeddings: return {}, []
    
    # Cluster
    norm_embeddings = np.array(embeddings) / np.linalg.norm(embeddings, axis=1, keepdims=True)
    clustering = DBSCAN(eps=0.18, min_samples=3, metric='cosine').fit(norm_embeddings)
    
    clusters = {}
    for path, label in zip(valid_paths, clustering.labels_):
        if label == -1: continue
        if label not in clusters: clusters[label] = []
        clusters[label].append(path)
        
    return clusters, valid_paths

In [29]:
import h5py
import gc
import uuid

# --- RECONSTRUCTION PIPELINE (H5 CLEAN REWRITE) ---
def run_reconstruction(images, dataset_name, scene_id):
    # Setup temporary directories
    scene_dir = Path(f"temp/{dataset_name}/{scene_id}")
    images_dir = scene_dir / "images"
    outputs_dir = scene_dir / "outputs"
    
    if scene_dir.exists(): shutil.rmtree(scene_dir)
    images_dir.mkdir(parents=True, exist_ok=True)
    outputs_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. SYMLINK & NAME MAP
    # We use safe names (img_0000.jpg) for everything to avoid path issues
    name_map = {}
    safe_images = []
    
    for i, p in enumerate(images):
        ext = p.suffix
        safe_name = f"img_{i:04d}{ext}"
        dest = images_dir / safe_name
        if not dest.exists():
            dest.symlink_to(p)
        name_map[safe_name] = p.name
        safe_images.append(safe_name)
    
    # 2. EXTRACT FEATURES (SIFT)
    unique_id = str(uuid.uuid4())[:8]
    feature_conf['output'] = f'feats_raw_{unique_id}'
    
    try:
        raw_feats_path = extract_features.main(feature_conf, images_dir, outputs_dir)
    except Exception as e:
        print(f"     ⚠️ Feature extraction failed: {e}")
        return None, {}

    # 3. REWRITE H5 FILE (CRITICAL FIX)
    # We create a new 'clean_feats.h5' that ONLY contains valid keys
    # This strips away any weird path prefixes hloc might have added
    clean_feats_path = outputs_dir / f"feats_clean_{unique_id}.h5"
    valid_images = []
    
    with h5py.File(str(raw_feats_path), 'r') as f_in, h5py.File(str(clean_feats_path), 'w') as f_out:
        for safe_name in safe_images:
            # hloc sometimes saves as "img_0000.jpg" or "images/img_0000.jpg"
            # We try to find the key
            key_found = None
            if safe_name in f_in: 
                key_found = safe_name
            elif f"images/{safe_name}" in f_in:
                key_found = f"images/{safe_name}"
            
            if key_found:
                f_in.copy(key_found, f_out, name=safe_name) # Save as JUST "img_0000.jpg"
                valid_images.append(safe_name)
            else:
                print(f"     ⚠️ Feature missing for {safe_name}")
                if (images_dir / safe_name).exists():
                    (images_dir / safe_name).unlink() # Delete bad image

    # Force close
    gc.collect()
    
    if len(valid_images) < 2: return None, {}

    # 4. PAIRS
    pairs_path = outputs_dir / "pairs.txt"
    pairs_from_exhaustive.main(pairs_path, image_list=valid_images)

    # 5. MATCHING (NN Mutual)
    matcher_conf['output'] = f'matches_{unique_id}'
    matches_path = outputs_dir / f"{matcher_conf['output']}.h5"
    
    try:
        # Match using the CLEAN features
        match_features.main(matcher_conf, pairs_path, features=clean_feats_path, matches=matches_path)
    except Exception as e:
        print(f"     ⚠️ Matching crashed: {e}")
        return None, {}
    
    gc.collect()

    # 6. RECONSTRUCTION
    try:
        model = reconstruction.main(
            outputs_dir / "sfm", images_dir, pairs_path, clean_feats_path, matches_path,
            camera_mode="SINGLE", image_options={"camera_model": "RADIAL"}
        )
        return model, name_map
    except Exception as e:
        print(f"     ⚠️ PyCOLMAP Reconstruction failed: {e}")
        return None, {}

In [30]:
# --- MAIN EXECUTION LOOP ---
INPUT_ROOT = Path("/kaggle/input/image-matching-challenge-2025/test")
submission_data = []

# Detect datasets (Folders inside test/)
datasets = [x for x in INPUT_ROOT.iterdir() if x.is_dir()]

print(f"Found {len(datasets)} datasets.")

for dataset in datasets:
    dataset_name = dataset.name
    print(f"\n=== Processing {dataset_name} ===")
    
    # 1. Cluster
    try:
        clusters, all_images = cluster_dataset(dataset)
    except:
        clusters, all_images = {}, []
        
    processed_images = set()
    
    # 2. Reconstruct
    for label, image_list in clusters.items():
        scene_name = f"scene_{label}"
        print(f"   > Reconstructing {scene_name} ({len(image_list)} images)...")
        
        # Run pipeline
        model, name_map = run_reconstruction(image_list, dataset_name, scene_name)
        
        if model:
            print(f"     ✅ Success! Registered {len(model.images)} images.")
            
            for img_id, img_obj in model.images.items():
                # Get the SAFE name (img_0000.png)
                safe_name = img_obj.name
                # Map back to ORIGINAL name
                original_name = name_map.get(safe_name, safe_name)
                
                R = img_obj.qvec2rotmat()
                T = img_obj.tvec
                r_str = ";".join(map(str, R.flatten()))
                t_str = ";".join(map(str, T.flatten()))
                
                submission_data.append({
                    "image_id": f"{dataset_name}_{original_name}",
                    "dataset": dataset_name,
                    "scene": scene_name,
                    "image": original_name,
                    "rotation_matrix": r_str,
                    "translation_vector": t_str
                })
                processed_images.add(original_name)
        else:
             print("     ⚠️ Reconstruction failed.")

    # 3. Handle Outliers
    for img_p in all_images:
        img_name = img_p.name
        if img_name not in processed_images:
            submission_data.append({
                "image_id": f"{dataset_name}_{img_name}",
                "dataset": dataset_name,
                "scene": "outliers",
                "image": img_name,
                "rotation_matrix": "nan;nan;nan;nan;nan;nan;nan;nan;nan",
                "translation_vector": "nan;nan;nan"
            })

# --- SAVE ---
df = pd.DataFrame(submission_data)
cols = ["image_id", "dataset", "scene", "image", "rotation_matrix", "translation_vector"]
if df.empty: df = pd.DataFrame(columns=cols)
else: df = df[cols].drop_duplicates(subset=["image_id"]).sort_values(["dataset", "scene"])

df.to_csv("submission.csv", index=False)
print(f"\n✅ Submission saved with {len(df)} rows.")
print(df.head())

Found 2 datasets.

=== Processing ETs ===
   Extracting DINO embeddings for 22 images...


[2025/12/04 20:31:36 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'dog',
           'options': {'first_octave': -1, 'peak_threshold': 0.01}},
 'output': 'feats_raw_e9048a8c',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2025/12/04 20:31:36 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'dog',
           'options': {'first_octave': -1, 'peak_threshold': 0.01}},
 'output': 'feats_raw_e9048a8c',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2025/12/04 20:31:36 hloc INFO] Found 10 images in root temp/ETs/scene_0/images.
[2025/12/04 20:31:36 hloc INFO] Found 10 images in root temp/ETs/scene_0/images.


   > Reconstructing scene_0 (10 images)...


100%|██████████| 10/10 [00:03<00:00,  2.70it/s]
[2025/12/04 20:31:40 hloc INFO] Finished exporting features.
[2025/12/04 20:31:40 hloc INFO] Finished exporting features.
[2025/12/04 20:31:40 hloc INFO] Found 45 pairs.
[2025/12/04 20:31:40 hloc INFO] Found 45 pairs.
[2025/12/04 20:31:40 hloc INFO] Matching local features with configuration:
{'model': {'distance_threshold': None,
           'do_mutual_check': True,
           'name': 'nearest_neighbor'},
 'output': 'matches_e9048a8c'}
[2025/12/04 20:31:40 hloc INFO] Matching local features with configuration:
{'model': {'distance_threshold': None,
           'do_mutual_check': True,
           'name': 'nearest_neighbor'},
 'output': 'matches_e9048a8c'}
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that e

     ⚠️ PyCOLMAP Reconstruction failed: 'Unable to synchronously open object (component not found)'
     ⚠️ Reconstruction failed.
   > Reconstructing scene_1 (9 images)...


100%|██████████| 9/9 [00:03<00:00,  2.50it/s]
[2025/12/04 20:31:45 hloc INFO] Finished exporting features.
[2025/12/04 20:31:45 hloc INFO] Finished exporting features.
[2025/12/04 20:31:45 hloc INFO] Found 36 pairs.
[2025/12/04 20:31:45 hloc INFO] Found 36 pairs.
[2025/12/04 20:31:45 hloc INFO] Matching local features with configuration:
{'model': {'distance_threshold': None,
           'do_mutual_check': True,
           'name': 'nearest_neighbor'},
 'output': 'matches_446d1758'}
[2025/12/04 20:31:45 hloc INFO] Matching local features with configuration:
{'model': {'distance_threshold': None,
           'do_mutual_check': True,
           'name': 'nearest_neighbor'},
 'output': 'matches_446d1758'}
 36%|███▌      | 13/36 [00:00<00:00, 28.29it/s]
Exception in thread Thread-161 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run


KeyboardInterrupt: 

    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/resource_sharer.py", line 86, in get_connection
 

In [20]:
# --- SAVE SUBMISSION ---
df = pd.DataFrame(submission_data)
cols = ["image_id", "dataset", "scene", "image", "rotation_matrix", "translation_vector"]

if df.empty:
    print("WARNING: No data generated. Creating empty submission.")
    df = pd.DataFrame(columns=cols)
else:
    df = df[cols]
    # Remove duplicates if any
    df.drop_duplicates(subset=["image_id"], keep="first", inplace=True)
    df.sort_values(["dataset", "scene"], inplace=True)

df.to_csv("submission.csv", index=False)
print("\n✅ Submission saved! Rows:", len(df))
print(df.head())


✅ Submission saved! Rows: 73
                           image_id dataset     scene  \
0  ETs_another_et_another_et001.png     ETs  outliers   
1  ETs_another_et_another_et002.png     ETs  outliers   
2  ETs_another_et_another_et003.png     ETs  outliers   
3  ETs_another_et_another_et004.png     ETs  outliers   
4  ETs_another_et_another_et005.png     ETs  outliers   

                          image                      rotation_matrix  \
0  another_et_another_et001.png  nan;nan;nan;nan;nan;nan;nan;nan;nan   
1  another_et_another_et002.png  nan;nan;nan;nan;nan;nan;nan;nan;nan   
2  another_et_another_et003.png  nan;nan;nan;nan;nan;nan;nan;nan;nan   
3  another_et_another_et004.png  nan;nan;nan;nan;nan;nan;nan;nan;nan   
4  another_et_another_et005.png  nan;nan;nan;nan;nan;nan;nan;nan;nan   

  translation_vector  
0        nan;nan;nan  
1        nan;nan;nan  
2        nan;nan;nan  
3        nan;nan;nan  
4        nan;nan;nan  
